In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv('Flat_Final_dataset_moscow_.csv')

In [3]:
df.head()

,ID ЖК,ЖК рус,Район Город,Округ Направление,Застройщик ЖК,Площадь,Тип Комнатности,Этаж,Дата регистрации,класс,Срок сдачи,Отделка,Зона,Стадия строительства в дату ДДУ,Цена кв. м,SalePrice
0,4983,Авиатика,59,3,Э.К. Девелопмент,58.6,1,22,2022-05-24,2,4 кв 2022,0,2,2,392820.0,23019252.0
1,4983,Авиатика,59,3,Э.К. Девелопмент,90.7,2,11,2022-05-26,2,4 кв 2022,0,2,2,368500.0,33422950.0
2,4983,Авиатика,59,3,Э.К. Девелопмент,105.5,3,19,2022-05-17,2,4 кв 2022,0,2,2,327900.0,34593450.0
3,4983,Авиатика,59,3,Э.К. Девелопмент,22.2,4,16,2022-05-17,2,4 кв 2022,0,2,2,509600.0,11313120.0
4,4983,Авиатика,59,3,Э.К. Девелопмент,88.8,2,15,2022-05-12,2,4 кв 2022,0,2,2,369600.0,32820480.0


In [4]:
X = df[['Район Город','класс', 'Отделка', 'Зона', 'Площадь', 'Тип Комнатности']]

In [5]:
X.head()

,Район Город,класс,Отделка,Зона,Площадь,Тип Комнатности
0,59,2,0,2,58.6,1
1,59,2,0,2,90.7,2
2,59,2,0,2,105.5,3
3,59,2,0,2,22.2,4
4,59,2,0,2,88.8,2


In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scaler = StandardScaler()

In [20]:
scaled_X = scaler.fit_transform(X.values)

In [9]:
scaled_df = pd.DataFrame(scaled_X, columns=X.columns)

In [10]:
scaled_df

,Район Город,класс,Отделка,Зона,Площадь,Тип Комнатности
0,1.400218,0.396203,-0.667816,-0.063438,0.386055,-0.249989
1,1.400218,0.396203,-0.667816,-0.063438,1.795231,0.453659
2,1.400218,0.396203,-0.667816,-0.063438,2.444945,1.157308
3,1.400218,0.396203,-0.667816,-0.063438,-1.211890,1.860956
4,1.400218,0.396203,-0.667816,-0.063438,1.711822,0.453659
...,...,...,...,...,...,...
22828,1.838015,0.396203,1.591851,-0.063438,-0.219759,-0.953637
22829,1.838015,0.396203,1.591851,-0.063438,-0.549006,-0.953637
22830,1.838015,0.396203,1.591851,-0.063438,-0.597296,-0.953637
22831,1.838015,0.396203,1.591851,-0.063438,1.180637,0.453659


In [11]:
from sklearn.cluster import DBSCAN

In [12]:
number_of_outliers = []

for eps in [0.15, 0.25, 0.5, 0.75]:
    model = DBSCAN(eps=eps)
    model.fit(scaled_df)

    number_of_outliers.append(np.sum(model.labels_ == -1))

KeyboardInterrupt: 

In [ ]:
sns.lineplot(x=[0.15, 0.25, 0.5, 0.75],y=number_of_outliers)
plt.ylabel("N of Points Classified as Outliers")
plt.xlabel("Epsilon Value")

In [14]:
model = DBSCAN(eps=0.5)
model.fit_predict(scaled_df)
labels = model.labels_
scaled_df['Labels'] = labels

In [16]:
scaled_df.values

array([[  1.40021781,   0.39620325,  -0.66781553, ...,   0.38605485,
         -0.24998883,   0.        ],
       [  1.40021781,   0.39620325,  -0.66781553, ...,   1.79523144,
          0.45365946,   1.        ],
       [  1.40021781,   0.39620325,  -0.66781553, ...,   2.44494527,
          1.15730776,   2.        ],
       ...,
       [  1.83801491,   0.39620325,   1.59185134, ...,  -0.5972958 ,
         -0.95363713, 144.        ],
       [  1.83801491,   0.39620325,   1.59185134, ...,   1.18063729,
          0.45365946, 171.        ],
       [  1.83801491,   0.39620325,   1.59185134, ...,  -0.57534601,
         -0.95363713, 144.        ]])

In [17]:
scaled_df = scaled_df.drop(scaled_df[scaled_df['Labels'] == -1].index)

In [18]:
from xgboost import XGBClassifier
classifier = XGBClassifier()

In [19]:
from sklearn.model_selection import GridSearchCV, train_test_split
X = scaled_df[['Район Город','класс', 'Отделка', 'Зона', 'Площадь', 'Тип Комнатности']]
y = scaled_df['Labels']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=101)

In [ ]:
X_test

,Район Город,класс,Отделка,Зона,Площадь,Тип Комнатности
3740,1.126595,0.396203,-0.667816,-0.063438,0.061198,-0.249989
12220,-0.460420,0.396203,-0.667816,1.901927,0.592383,0.453659
18935,-0.788768,-1.428609,1.591851,-2.028803,0.934800,-0.249989
20239,0.634073,0.396203,-0.667816,-0.063438,0.456294,-0.249989
13387,0.251000,0.396203,-0.667816,-0.063438,-0.347068,-0.953637
...,...,...,...,...,...,...
14955,1.345493,-1.428609,1.591851,-0.063438,-0.048551,-0.953637
7327,0.634073,0.396203,1.591851,-0.063438,-0.856303,1.860956
21358,0.360450,0.396203,1.591851,-0.063438,1.044549,0.453659
8664,-0.296246,0.396203,1.591851,-0.063438,-0.303169,-0.249989


In [ ]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
print(accuracy_score(y_test, pred))

0.9940775836541309


In [24]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       458
           1       0.99      1.00      1.00       165
           2       1.00      0.75      0.86         4
           3       1.00      1.00      1.00       276
           4       1.00      1.00      1.00       550
           5       1.00      1.00      1.00        87
           6       0.99      1.00      0.99        93
           7       0.97      1.00      0.99        39
           8       1.00      0.94      0.97        17
           9       1.00      1.00      1.00        42
          10       0.93      0.93      0.93        15
          11       1.00      1.00      1.00         1
          12       0.93      1.00      0.97        14
          13       1.00      1.00      1.00        96
          14       0.92      0.97      0.95        36
          15       1.00      1.00      1.00        62
          16       1.00      1.00      1.00       105
          17       1.00    

c:\Files\Вуз\Прога и алгоритмы\OOP_project\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Files\Вуз\Прога и алгоритмы\OOP_project\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Files\Вуз\Прога и алгоритмы\OOP_project\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [22]:
from joblib import dump

In [26]:
dump(classifier, 'Classifier_DBSCAN.joblib')

['Classifier_DBSCAN.joblib']

In [27]:
scaled_df.to_csv("Scaled_df_for_classifier.csv",index=False)

In [23]:
dump(scaler, 'scaler_for_dbscan.gz')

['scaler_for_dbscan.gz']